# SIMPLE AND QUICK IBNR CALCULATIONS IN PYTHON USING CL LIBRARY

In [ ]:
import numpy as np
import pandas as pd
import chainladder as cl
import matplotlib.pyplot as plt
%matplotlib inline

Import claims data. This example uses claims paid data. The dates in the csv file originally came from a Microsoft Excel file. As such, there is a subtraction of 2 days in order to obtain correct date information - see <a href = 'https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjLytv2lqyBAxU8TEEAHVz0DOYQFnoECA0QAw&url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FLeap_year_problem%23%3A~%3Atext%3DOccurrences%2C-See%2520also%253A%2520Year%26text%3DMicrosoft%2520Excel%2520has%252C%2520since%2520its%2Cthe%2520purpose%2520of%2520backward%2520compatibility.&usg=AOvVaw1nf4EsIyXXEVS_BNUu2qkR&opi=89978449'>Excel Leap Year Bug</a>

In [ ]:
data = pd.read_csv('Claims Data.csv')

for col in list(data.columns):
    if 'Unnamed' in col:
        data = data.drop(col, axis = 1)
        
origin = np.datetime64('1900-01-01', 'D')

data['MAIN CLASS'] = data['MAIN CLASS'].astype(str)

data['LOSS DATE'] = data['LOSS DATE'] - 2
data['LOSS DATE'] = origin + np.array(data['LOSS DATE'], dtype= 'timedelta64[D]')

data['PAID DATE'] = data['PAID DATE'] - 2
data['PAID DATE'] = origin + np.array(data['PAID DATE'], dtype = 'timedelta64[D]')

data = data[data["LOSS DATE"] > '2016-06-30']

# subset dataframes by Class
# ['GMS', 'MEDICAL', 'MOTOR', 'GPA', 'EN', 'LIA', 'MIS', 'PTY', 'TRN', 'TRAVEL EASY INSURANCE']

classes_of_business = data['MAIN CLASS'].unique()
df_dict = {classes_of_business[i]: data[data['MAIN CLASS'] == classes_of_business[i]] for i in range(len(classes_of_business))}

# subset dataframes allow for extraction of individual class data for easier analysis with pandas if the need arises. 

Create Gross and Net triangles. This can be done in a single block of code by providing a list ['GROSS AMOUNT', 'NET AMOUNT'], to the columns parameter. The code below spilts this for more granular analysis when referencing individual classes and the triangulation approach to be taken. cumulative parameter can be set to True from the onset to obtain cumulative triangles without the need for the incr_to_cum() method. However, this has not been done in order to allow for visibility of incremental triangles. OYDM grain has been selected to illustrate traigulation with Origin Year and Monthly Development. 

In [ ]:
all_gross_triangles = {cob: cl.Triangle(
        data = df_dict[cob],
        origin = 'LOSS DATE',
        development = 'PAID DATE',
        columns = 'GROSS AMOUNT',
        cumulative = False
    ) for cob in classes_of_business }

all_net_triangles = {cob: cl.Triangle(
        data = df_dict[cob],
        origin = 'LOSS DATE',
        development = 'PAID DATE',
        columns = 'NET AMOUNT',
        cumulative = False
    ) for cob in classes_of_business }

my_classes = ["MOTOR", "LIA", "GPA", "GMS", "MY CARE"]
for cob in all_gross_triangles.keys():
    if cob in my_classes:
        cl_triangle = all_gross_triangles[cob].grain('OYDM')
        cl_triangle = cl_triangle.incr_to_cum()
        tri_df = cl_triangle.to_frame()
        tri_df.to_csv(f"Gross Triangles/{cob}_OYDM_gross_triangle.csv", index = True)

for cob in all_net_triangles.keys():
    if cob in my_classes:
        cl_triangle = all_net_triangles[cob].grain('OYDM')
        cl_triangle = cl_triangle.incr_to_cum()
        tri_df = cl_triangle.to_frame()
        tri_df.to_csv(f"Net Triangles/{cob}_OYDM_net_triangle.csv", index = True)


We illustrate analysis of the motor class below. The triangle grain has been changed to OYDY for the below to demonstrate quick and flexible transition between triangle grains provided by the CL library.

In [ ]:
motor = all_net_triangles['MOTOR'].grain('OYDY')
motor = motor.incr_to_cum()
motor.link_ratio

The IBNR follows simply by the below code. The library offers Bornhuetter-Ferguson, Cape-Cop and Mack Chainladder options among others for the IBNR fit. These require their own additional inputs (premiums and loss ratios) to produce results. The initial dataset was paid claims data and therefore the IBNR numbers resulting from the below code will be IBNR + IBNER. It is the individual's responsibility to account for case reserves to obtain the pure IBNR.

In [ ]:
motor = cl.Chainladder().fit(motor)
motor_reserve = motor.ibnr_
motor_reserve